# Copyright Netherlands eScience Center <br>
** Function     : Packing netCDF for the radiation fields (TOA & surface) from ERA-Interim** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2018.09.09 ** <br>
** Last Update  : 2018.09.09 ** <br>
Description     : This notebook aims to pack the TOA/surface fields of radiations from ERA-Interim.<br>
Return Values   : netCDF4 <br>
Caveat          : The radiations fields are forecast fields instead of analysis fields. It is accumulated from a certain forecast time and thus for the values at certain time we should subtract the accumulated values. The prediction is 12 hrs. The starting time for each prediction as well as the predicted time are given below: <br>
00:00 : 3:00 + 6:00 + 9:00 + 12:00 <br>
12:00 : 3:00 + 6:00 + 9:00 + 12:00 <br>

In [1]:
import numpy as np
import scipy as sp
import time as tttt
from netCDF4 import Dataset,num2date
import os

In [2]:
################################   Input zone  ######################################
# specify starting and ending time
start_year = 1979
end_year = 2017
# specify data path
# ERAI 3D fields on pressure level
datapath_radiation = '/home/yang/workbench/Core_Database_AMET_OMET_reanalysis/ERAI/regression/surface/radiation'
# specify output path for figures
output_path = '/home/yang/workbench/Core_Database_AMET_OMET_reanalysis/ERAI/regression'
####################################################################################

In [3]:
def var_key_retrieve(datapath, year):
    # get the path to each datasets
    print ("Start retrieving datasets %d (y)" % (year))
    # The shape of each variable is (241,480)
    datapath_full = datapath + os.sep + 'surface_erai_monthly_075_%d_radiation.nc' % (year)
    # get the variable keys
    var_key = Dataset(datapath_full)

    print ("Retrieving datasets successfully and return the variable key!")
    return var_key

In [4]:
def create_netcdf_point (pool_sshf, pool_slhf, pool_ssr, pool_str, pool_tsr, pool_ttr, output_path):
    print ('*******************************************************************')
    print ('*********************** create netcdf file*************************')
    print ('*******************************************************************')
    #logging.info("Start creating netcdf file for the 2D fields of ERAI at each grid point.")
    # wrap the datasets into netcdf file
    # 'NETCDF3_CLASSIC', 'NETCDF3_64BIT', 'NETCDF4_CLASSIC', and 'NETCDF4'
    data_wrap = Dataset(output_path + os.sep + 'surface_erai_monthly_regress_1979_2017_radiation.nc','w',format = 'NETCDF4')
    # create dimensions for netcdf data
    year_wrap_dim = data_wrap.createDimension('year',Dim_year)
    month_wrap_dim = data_wrap.createDimension('month',Dim_month)
    lat_wrap_dim = data_wrap.createDimension('latitude',Dim_latitude)
    lon_wrap_dim = data_wrap.createDimension('longitude',Dim_longitude)
    # create coordinate variable
    year_wrap_var = data_wrap.createVariable('year',np.int32,('year',))
    month_wrap_var = data_wrap.createVariable('month',np.int32,('month',))
    lat_wrap_var = data_wrap.createVariable('latitude',np.float32,('latitude',))
    lon_wrap_var = data_wrap.createVariable('longitude',np.float32,('longitude',))
    # create the actual 4d variable
    sshf_wrap_var = data_wrap.createVariable('sshf',np.float64,('year','month','latitude','longitude'),zlib=True)
    slhf_wrap_var = data_wrap.createVariable('slhf',np.float64,('year','month','latitude','longitude'),zlib=True)
    ssr_wrap_var = data_wrap.createVariable('ssr',np.float64,('year','month','latitude','longitude'),zlib=True)
    str_wrap_var = data_wrap.createVariable('str',np.float64,('year','month','latitude','longitude'),zlib=True)
    tsr_wrap_var = data_wrap.createVariable('tsr',np.float64,('year','month','latitude','longitude'),zlib=True)
    ttr_wrap_var = data_wrap.createVariable('ttr',np.float64,('year','month','latitude','longitude'),zlib=True)    
    # global attributes
    data_wrap.description = 'Monthly mean 2D fields of ERA-Interim on surface level'
    # variable attributes
    lat_wrap_var.units = 'degree_north'
    lon_wrap_var.units = 'degree_east'

    sshf_wrap_var.units = 'W/m2'
    slhf_wrap_var.units = 'W/m2'
    ssr_wrap_var.units = 'W/m2'
    str_wrap_var.units = 'W/m2'
    tsr_wrap_var.units = 'W/m2'
    ttr_wrap_var.units = 'W/m2'

    sshf_wrap_var.long_name = 'surface sensible heat flux'
    slhf_wrap_var.long_name = 'surface latent heat flux'
    ssr_wrap_var.long_name = 'surface net solar radiation'
    str_wrap_var.long_name = 'surface net thermal radiation'
    tsr_wrap_var.long_name = 'top net solar radiation'
    ttr_wrap_var.long_name = 'top net thermal radiation'
    

    # writing data
    lat_wrap_var[:] = latitude
    lon_wrap_var[:] = longitude
    month_wrap_var[:] = index_month
    year_wrap_var[:] = period

    sshf_wrap_var[:] = pool_sshf
    slhf_wrap_var[:] = pool_slhf
    ssr_wrap_var[:] = pool_ssr
    str_wrap_var[:] = pool_str
    tsr_wrap_var[:] = pool_tsr
    ttr_wrap_var[:] = pool_ttr

    # close the file
    data_wrap.close()
    print ("The generation of netcdf files for fields on surface is complete!!")

In [5]:
def retriver(key):
    print ('Extract synoptic mean fields.')
    sshf_accumulate = var_key.variables['sshf'][:]
    slhf_accumulate = var_key.variables['slhf'][:]
    ssr_accumulate = var_key.variables['ssr'][:]
    str_accumulate = var_key.variables['str'][:]
    tsr_accumulate = var_key.variables['tsr'][:]
    ttr_accumulate = var_key.variables['ttr'][:]
    # create arrays to store the values after removing accumulation
    sshf_synoptic = np.zeros(sshf_accumulate.shape)
    slhf_synoptic = np.zeros(sshf_accumulate.shape)
    ssr_synoptic = np.zeros(sshf_accumulate.shape)
    str_synoptic = np.zeros(sshf_accumulate.shape)
    tsr_synoptic = np.zeros(sshf_accumulate.shape)
    ttr_synoptic = np.zeros(sshf_accumulate.shape)
    # remove the accumulation and take the monthly mean
    sshf_synoptic[0::4,:,:] = sshf_accumulate[0::4,:,:]
    slhf_synoptic[0::4,:,:] = slhf_accumulate[0::4,:,:]
    ssr_synoptic[0::4,:,:] = ssr_accumulate[0::4,:,:]
    str_synoptic[0::4,:,:] = str_accumulate[0::4,:,:]
    tsr_synoptic[0::4,:,:] = tsr_accumulate[0::4,:,:]
    ttr_synoptic[0::4,:,:] = ttr_accumulate[0::4,:,:]
    for i in np.arange(3):
        sshf_synoptic[i+1::4,:,:] = sshf_accumulate[i+1::4,:,:] - sshf_accumulate[i::4,:,:]
        slhf_synoptic[i+1::4,:,:] = slhf_accumulate[i+1::4,:,:] - slhf_accumulate[i::4,:,:]
        ssr_synoptic[i+1::4,:,:] = ssr_accumulate[i+1::4,:,:] - ssr_accumulate[i::4,:,:]
        str_synoptic[i+1::4,:,:] = str_accumulate[i+1::4,:,:] - str_accumulate[i::4,:,:]
        tsr_synoptic[i+1::4,:,:] = tsr_accumulate[i+1::4,:,:] - tsr_accumulate[i::4,:,:]
        ttr_synoptic[i+1::4,:,:] = ttr_accumulate[i+1::4,:,:] - ttr_accumulate[i::4,:,:]
    # create the arrays for monthly mean
    lat = var_key.variables['latitude'][:]
    lon = var_key.variables['longitude'][:]
    sshf_monthly = np.zeros((12, len(lat), len(lon)),dtype=float)
    slhf_monthly = np.zeros((12, len(lat), len(lon)),dtype=float)
    ssr_monthly = np.zeros((12, len(lat), len(lon)),dtype=float)
    str_monthly = np.zeros((12, len(lat), len(lon)),dtype=float)
    tsr_monthly = np.zeros((12, len(lat), len(lon)),dtype=float)
    ttr_monthly = np.zeros((12, len(lat), len(lon)),dtype=float)
    # take the mean per month and change the unit to W/m2
    for i in np.arange(12):
        sshf_monthly[i,:,:] = np.mean(sshf_synoptic[i*8:i*8+8,:,:], 0) / (3 * 3600)
        slhf_monthly[i,:,:] = np.mean(slhf_synoptic[i*8:i*8+8,:,:], 0) / (3 * 3600)
        ssr_monthly[i,:,:] = np.mean(ssr_synoptic[i*8:i*8+8,:,:], 0) / (3 * 3600)
        str_monthly[i,:,:] = np.mean(str_synoptic[i*8:i*8+8,:,:], 0) / (3 * 3600)
        tsr_monthly[i,:,:] = np.mean(tsr_synoptic[i*8:i*8+8,:,:], 0) / (3 * 3600)
        ttr_monthly[i,:,:] = np.mean(ttr_synoptic[i*8:i*8+8,:,:], 0) / (3 * 3600)
    return sshf_monthly, slhf_monthly, ssr_monthly, str_monthly, tsr_monthly, ttr_monthly

In [6]:
if __name__=="__main__":
    ####################################################################
    ######  Create time namelist matrix for variable extraction  #######
    ####################################################################
    # date and time arrangement
    # namelist of month and days for file manipulation
    namelist_month = ['01','02','03','04','05','06','07','08','09','10','11','12']
    # index of months
    period = np.arange(start_year,end_year+1,1)
    index_month = np.arange(1,13,1)
    ####################################################################
    ######       Extract invariant and calculate constants       #######
    ####################################################################
    # get invariant from benchmark file
    Dim_year = len(period)
    Dim_month = len(index_month)
    Dim_latitude = 241
    Dim_longitude = 480
    #############################################
    #####   Create space for stroing data   #####
    #############################################
    # data pool
    pool_sshf = np.zeros((Dim_year,Dim_month,Dim_latitude,Dim_longitude),dtype = float)
    pool_slhf = np.zeros((Dim_year,Dim_month,Dim_latitude,Dim_longitude),dtype = float)
    pool_ssr = np.zeros((Dim_year,Dim_month,Dim_latitude,Dim_longitude),dtype = float)
    pool_str = np.zeros((Dim_year,Dim_month,Dim_latitude,Dim_longitude),dtype = float)
    pool_tsr = np.zeros((Dim_year,Dim_month,Dim_latitude,Dim_longitude),dtype = float)
    pool_ttr = np.zeros((Dim_year,Dim_month,Dim_latitude,Dim_longitude),dtype = float)
    latitude = np.zeros(Dim_latitude,dtype=float)
    longitude = np.zeros(Dim_longitude,dtype=float)
    # loop for calculation
    for i in period:
        # get the key of each variable
        var_key = var_key_retrieve(datapath_radiation,i)
        latitude = var_key.variables['latitude'][:]
        longitude = var_key.variables['longitude'][:]
        sshf_monthly, slhf_monthly, ssr_monthly, str_monthly,\
        tsr_monthly, ttr_monthly = retriver(var_key)
        pool_sshf[i-1979,:,:,:] = sshf_monthly
        pool_slhf[i-1979,:,:,:] = slhf_monthly
        pool_ssr[i-1979,:,:,:] = ssr_monthly
        pool_str[i-1979,:,:,:] = str_monthly
        pool_tsr[i-1979,:,:,:] = tsr_monthly
        pool_ttr[i-1979,:,:,:] = ttr_monthly
    ####################################################################
    ######                 Data Wrapping (NetCDF)                #######
    ####################################################################
    create_netcdf_point(pool_sshf, pool_slhf, pool_ssr, pool_str,
                        pool_tsr, pool_ttr, output_path)
    print ('Packing 2D fields of ERA-Interim on surface level is complete!!!')
    print ('The output is in sleep, safe and sound!!!')

Start retrieving datasets 1979 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1980 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1981 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1982 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1983 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1984 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1985 (y)
Retrieving datasets successfully and return the variable key!
Extract synoptic mean fields.
Start retrieving datasets 1986 (y)
Retrieving datasets successfully and return the variable key!
Extract synopt